In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = SparkConf().setAppName('myApp') \
.setMaster('local')
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

In [2]:
from pyspark.sql.functions import *
import pandas as pd
import math

In [3]:
df_application = spark.read.format('csv').options(header=True,inferSchema=True).load('application_record.csv')
df_application.show(30, truncate=50)
df_application.printSchema()

+-------+-----------+------------+---------------+------------+----------------+--------------------+-----------------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
|     ID|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|    NAME_INCOME_TYPE|          NAME_EDUCATION_TYPE|  NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|DAYS_BIRTH|DAYS_EMPLOYED|FLAG_MOBIL|FLAG_WORK_PHONE|FLAG_PHONE|FLAG_EMAIL|OCCUPATION_TYPE|CNT_FAM_MEMBERS|
+-------+-----------+------------+---------------+------------+----------------+--------------------+-----------------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
|5008804|          M|           Y|              Y|           0|        427500.0|             Working|             Higher education|      Civil marriage| Rented apartment

In [4]:
# Check for null etc.
df_application.select([count((when(isnan(c) | isnull(c), 1))).alias(c) for c in df_application.columns]).show()
# No null in data so all data is valid

+---+-----------+------------+---------------+------------+----------------+----------------+-------------------+------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
| ID|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|NAME_INCOME_TYPE|NAME_EDUCATION_TYPE|NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|DAYS_BIRTH|DAYS_EMPLOYED|FLAG_MOBIL|FLAG_WORK_PHONE|FLAG_PHONE|FLAG_EMAIL|OCCUPATION_TYPE|CNT_FAM_MEMBERS|
+---+-----------+------------+---------------+------------+----------------+----------------+-------------------+------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
|  0|          0|           0|              0|           0|               0|               0|                  0|                 0|                0|         0|            0|         0|              0|         0|         0|     

In [5]:
# After checking the flag_mobil is not important because the data is only 1 and no other variation. so in the cleaned data not getting the flag mobil
df_app_cleaned = df_application.select(['id','code_gender','flag_own_car','flag_own_realty','cnt_children','amt_income_total',
                                        'name_income_type','name_education_type','name_family_status','name_housing_type','days_birth',
                                        'days_employed','flag_work_phone','flag_phone','flag_email','occupation_type','cnt_fam_members'])

# AND the value is always 0 an 1 as the boolean so we want to make our data uniform so changing the Y as 1 and N as 0, F as 1 and M as 0
df_app_cleaned = df_app_cleaned.withColumn('code_gender', when(df_app_cleaned.code_gender == 'F',1).otherwise(0))
df_app_cleaned = df_app_cleaned.withColumn('flag_own_car', when(df_app_cleaned.flag_own_car == 'Y',1).otherwise(0))
df_app_cleaned = df_app_cleaned.withColumn('flag_own_realty', when(df_app_cleaned.flag_own_car == 'Y',1).otherwise(0))

# More data cleaning cnt_fam_members should be integer because there shouldn't be fam members that is half or less than 1
df_app_cleaned = df_app_cleaned.withColumn('cnt_fam_members', col('cnt_fam_members').cast('Integer'))

df_app_cleaned.show()

+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+---------------+----------+----------+---------------+---------------+
|     id|code_gender|flag_own_car|flag_own_realty|cnt_children|amt_income_total|    name_income_type| name_education_type|  name_family_status|name_housing_type|days_birth|days_employed|flag_work_phone|flag_phone|flag_email|occupation_type|cnt_fam_members|
+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+---------------+----------+----------+---------------+---------------+
|5008804|          0|           1|              0|           0|        427500.0|             Working|    Higher education|      Civil marriage| Rented apartment|    -12005|        -4542|              1|         0|         0|     

In [6]:
# Changing the days birth into age Integer so that it will be classification into its correspondence age
df_app_cleaned = df_app_cleaned.withColumn('age', floor(col('days_birth')/365.25) * -1)

# Check value because age should not be negative or even > 100
df_app_cleaned.select('age').where('age < 0 or age > 100').show()

# Changing the days birth into age Integer so that it will be classification into its correspondence age
df_app_cleaned = df_app_cleaned.withColumn('employed', floor(col('days_employed')/365.25) * -1)

# Check value because age should not be negative or even > 100 as well as the occupation
df_app_cleaned.select(['name_income_type','occupation_type','employed']).where('employed < 0 or employed > 100').distinct().show()

+---+
|age|
+---+
+---+

+----------------+---------------+--------+
|name_income_type|occupation_type|employed|
+----------------+---------------+--------+
|       Pensioner|           NULL|    -999|
+----------------+---------------+--------+



In [7]:
# because the data is said pensioner than it is possible that the employed is negative but for calculation sake it is better to make it into 0
df_app_cleaned = df_app_cleaned.withColumn('employed', when(df_app_cleaned.employed < 0,0).otherwise(df_app_cleaned.employed))

df_app_cleaned.show()

+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+---------------+----------+----------+---------------+---------------+---+--------+
|     id|code_gender|flag_own_car|flag_own_realty|cnt_children|amt_income_total|    name_income_type| name_education_type|  name_family_status|name_housing_type|days_birth|days_employed|flag_work_phone|flag_phone|flag_email|occupation_type|cnt_fam_members|age|employed|
+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+---------------+----------+----------+---------------+---------------+---+--------+
|5008804|          0|           1|              0|           0|        427500.0|             Working|    Higher education|      Civil marriage| Rented apartment|    -12005|        -4542|    

In [8]:
# there is occupation type that is null so we need to fill it up
df_app_cleaned.select('occupation_type').where(isnull(col('occupation_type'))).count() 
# There are no possibility to

134203

In [9]:
# check for occupation type that is null
df_app_cleaned.select(['amt_income_total','name_income_type','occupation_type','employed']).where(isnull(df_app_cleaned.occupation_type)).show()
# there is possibility to use the amt_income_total & name_income_type to classified the occupation_type typically will have the same job
# so need to find the exact number to it

+----------------+----------------+---------------+--------+
|amt_income_total|name_income_type|occupation_type|employed|
+----------------+----------------+---------------+--------+
|        427500.0|         Working|           NULL|      13|
|        427500.0|         Working|           NULL|      13|
|        283500.0|       Pensioner|           NULL|       0|
|        283500.0|       Pensioner|           NULL|       0|
|        283500.0|       Pensioner|           NULL|       0|
|        112500.0|         Working|           NULL|       5|
|        112500.0|         Working|           NULL|       5|
|        112500.0|         Working|           NULL|       5|
|        315000.0|       Pensioner|           NULL|       0|
|        225000.0|         Working|           NULL|       8|
|        225000.0|         Working|           NULL|       8|
|        225000.0|         Working|           NULL|       8|
|        225000.0|         Working|           NULL|       8|
|        157500.0|      

In [10]:
# so use the possibility of it and fill the occupation_type that got from the amt_income_total and name_income_type
from pyspark.sql import Window, functions as F

# Define a window specification
window_spec = Window.partitionBy('amt_income_total', 'name_income_type')

# Fill missing values of occupation_type with the most frequent value within each group
filled_occupation_type = F.first('occupation_type', ignorenulls=True).over(window_spec)

# Fill missing values using coalesce
df_app_cleaned = df_app_cleaned.withColumn('filled_occupation_type', filled_occupation_type)
df_app_cleaned = df_app_cleaned.withColumn('occupation_type', F.coalesce('occupation_type', 'filled_occupation_type')).drop('filled_occupation_type')

df_app_cleaned.show()

+-------+-----------+------------+---------------+------------+----------------+----------------+--------------------+--------------------+-----------------+----------+-------------+---------------+----------+----------+---------------+---------------+---+--------+
|     id|code_gender|flag_own_car|flag_own_realty|cnt_children|amt_income_total|name_income_type| name_education_type|  name_family_status|name_housing_type|days_birth|days_employed|flag_work_phone|flag_phone|flag_email|occupation_type|cnt_fam_members|age|employed|
+-------+-----------+------------+---------------+------------+----------------+----------------+--------------------+--------------------+-----------------+----------+-------------+---------------+----------+----------+---------------+---------------+---+--------+
|6093713|          1|           0|              0|           0|         26100.0|       Pensioner|Secondary / secon...|             Married|House / apartment|    -21003|       365243|              0|    

In [11]:
# check for occupation type that is null  and that the amt_income_total and name_income_type same, but still null
df_app_cleaned.select(['amt_income_total','name_income_type','occupation_type','employed']).where(isnull(df_app_cleaned.occupation_type) & (df_app_cleaned.employed != 0)).show()
# There are no exact data to fill the null in that missing value, so the possibility is the same name_income_type with the nearest amt_income_total
# Create a window specification to partition by 'name_income_type' and order by the absolute difference in 'amt_income_total'
window_spec = Window.partitionBy('name_income_type').orderBy(F.abs(F.col('amt_income_total') - F.col('amt_income_total_with_null')))

# Create a temporary column to store the amt_income_total of rows with null occupation_type
df_temp = df_app_cleaned.withColumn('amt_income_total_with_null', F.when(F.col('occupation_type').isNull(), F.col('amt_income_total')))

# Fill missing occupation_type using the nearest amt_income_total
filled_occupation_type = F.first('occupation_type', ignorenulls=True).over(window_spec)
df_temp = df_temp.withColumn('filled_occupation_type', filled_occupation_type)

# Use coalesce to fill the null values
df_app_cleaned = df_temp.withColumn('occupation_type', F.coalesce(F.col('occupation_type'), F.col('filled_occupation_type'))).drop('filled_occupation_type', 'amt_income_total_with_null')

df_app_cleaned.show()

+----------------+--------------------+---------------+--------+
|amt_income_total|    name_income_type|occupation_type|employed|
+----------------+--------------------+---------------+--------+
|         32400.0|             Working|           NULL|       5|
|         32400.0|             Working|           NULL|       5|
|         32400.0|             Working|           NULL|       5|
|         38250.0|       State servant|           NULL|       3|
|         38250.0|       State servant|           NULL|       3|
|         38250.0|       State servant|           NULL|       3|
|         38250.0|       State servant|           NULL|       3|
|         38250.0|       State servant|           NULL|       3|
|         38250.0|       State servant|           NULL|       3|
|         38250.0|       State servant|           NULL|       3|
|         38700.0|             Working|           NULL|      26|
|         41211.0|Commercial associate|           NULL|       6|
|         41211.0|Commerc

In [12]:
# check for occupation type that is null
df_app_cleaned.select(['amt_income_total','name_income_type','occupation_type','employed']).where(isnull(df_app_cleaned.occupation_type)).show()
# Data already cleaned!!

+----------------+----------------+---------------+--------+
|amt_income_total|name_income_type|occupation_type|employed|
+----------------+----------------+---------------+--------+
+----------------+----------------+---------------+--------+



In [13]:
df_credit = spark.read.format('csv').options(header=True,inferSchema=True).load('credit_record.csv')
df_credit.show(30, truncate=50)
df_credit.printSchema()
print('Status \n\
      0 => 1-29 days past due \n\
      1 => 30-59 days past due \n\
      2 => 60-89 days past due \n\
      3 => 90-119 days past due \n\
      4 => 120-149 days past due \n\
      5 => overdue/ bad debts > 150 days \n\
      C => paid successfully \n\
      X => No loan \n\
')

+-------+--------------+------+
|     ID|MONTHS_BALANCE|STATUS|
+-------+--------------+------+
|5001711|             0|     X|
|5001711|            -1|     0|
|5001711|            -2|     0|
|5001711|            -3|     0|
|5001712|             0|     C|
|5001712|            -1|     C|
|5001712|            -2|     C|
|5001712|            -3|     C|
|5001712|            -4|     C|
|5001712|            -5|     C|
|5001712|            -6|     C|
|5001712|            -7|     C|
|5001712|            -8|     C|
|5001712|            -9|     0|
|5001712|           -10|     0|
|5001712|           -11|     0|
|5001712|           -12|     0|
|5001712|           -13|     0|
|5001712|           -14|     0|
|5001712|           -15|     0|
|5001712|           -16|     0|
|5001712|           -17|     0|
|5001712|           -18|     0|
|5001713|             0|     X|
|5001713|            -1|     X|
|5001713|            -2|     X|
|5001713|            -3|     X|
|5001713|            -4|     X|
|5001713

In [14]:
# Check for null etc.
df_credit.select([count((when(isnan(c) | isnull(c), 1))).alias(c) for c in df_credit.columns]).show()
# No null in data so all data is valid# Check for null etc.
df_credit.select([count((when(isnan(c) | isnull(c), 1))).alias(c) for c in df_credit.columns]).show()
# No null in data so all data is valid

+---+--------------+------+
| ID|MONTHS_BALANCE|STATUS|
+---+--------------+------+
|  0|             0|     0|
+---+--------------+------+

+---+--------------+------+
| ID|MONTHS_BALANCE|STATUS|
+---+--------------+------+
|  0|             0|     0|
+---+--------------+------+



In [15]:
df_credit.show()

+-------+--------------+------+
|     ID|MONTHS_BALANCE|STATUS|
+-------+--------------+------+
|5001711|             0|     X|
|5001711|            -1|     0|
|5001711|            -2|     0|
|5001711|            -3|     0|
|5001712|             0|     C|
|5001712|            -1|     C|
|5001712|            -2|     C|
|5001712|            -3|     C|
|5001712|            -4|     C|
|5001712|            -5|     C|
|5001712|            -6|     C|
|5001712|            -7|     C|
|5001712|            -8|     C|
|5001712|            -9|     0|
|5001712|           -10|     0|
|5001712|           -11|     0|
|5001712|           -12|     0|
|5001712|           -13|     0|
|5001712|           -14|     0|
|5001712|           -15|     0|
+-------+--------------+------+
only showing top 20 rows



In [21]:
df_credit = df_credit.filter(col("MONTHS_BALANCE") >= -6)
df_credit.show()

+-------+--------------+------+
|     ID|MONTHS_BALANCE|STATUS|
+-------+--------------+------+
|5001711|             0|     X|
|5001711|            -1|     0|
|5001711|            -2|     0|
|5001711|            -3|     0|
|5001712|             0|     C|
|5001712|            -1|     C|
|5001712|            -2|     C|
|5001712|            -3|     C|
|5001712|            -4|     C|
|5001712|            -5|     C|
|5001712|            -6|     C|
|5001713|             0|     X|
|5001713|            -1|     X|
|5001713|            -2|     X|
|5001713|            -3|     X|
|5001713|            -4|     X|
|5001713|            -5|     X|
|5001713|            -6|     X|
|5001714|             0|     X|
|5001714|            -1|     X|
+-------+--------------+------+
only showing top 20 rows



In [22]:
df_app_cleaned.show()

+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+------------------+-----------------+----------+-------------+---------------+----------+----------+---------------+---------------+---+--------+
|     id|code_gender|flag_own_car|flag_own_realty|cnt_children|amt_income_total|    name_income_type| name_education_type|name_family_status|name_housing_type|days_birth|days_employed|flag_work_phone|flag_phone|flag_email|occupation_type|cnt_fam_members|age|employed|
+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+------------------+-----------------+----------+-------------+---------------+----------+----------+---------------+---------------+---+--------+
|6014233|          1|           0|              0|           1|         28723.5|Commercial associate|Secondary / secon...|           Married|House / apartment|    -14361|         -138|            

In [23]:
df_app = df_app_cleaned \
    .withColumnRenamed('CODE_GENDER', 'gender') \
    .withColumnRenamed('FLAG_OWN_CAR', 'own_car') \
    .withColumnRenamed('FLAG_OWN_REALTY', 'own_property') \
    .withColumnRenamed('CNT_CHILDREN', 'children') \
    .withColumnRenamed('AMT_INCOME_TOTAL', 'income') \
    .withColumnRenamed('NAME_INCOME_TYPE', 'income_type') \
    .withColumnRenamed('NAME_EDUCATION_TYPE', 'education') \
    .withColumnRenamed('NAME_FAMILY_STATUS', 'family_status') \
    .withColumnRenamed('NAME_HOUSING_TYPE', 'housing_type') \
    .withColumnRenamed('FLAG_MOBIL', 'mobile') \
    .withColumnRenamed('FLAG_WORK_PHONE', 'work_phone') \
    .withColumnRenamed('FLAG_PHONE', 'phone') \
    .withColumnRenamed('FLAG_EMAIL', 'email') \
    .withColumnRenamed('CNT_FAM_MEMBERS', 'family_members') \
    .withColumnRenamed('MONTHS_BALANCE', 'months_balance') \
    .withColumnRenamed('STATUS', 'status') \
    .withColumnRenamed('DAYS_BIRTH', 'age_in_days') \
    .withColumnRenamed('DAYS_EMPLOYED', 'employment_in_days')

# Show the DataFrame schema to confirm changes
df_app.printSchema()

root
 |-- id: integer (nullable = true)
 |-- gender: integer (nullable = false)
 |-- own_car: integer (nullable = false)
 |-- own_property: integer (nullable = false)
 |-- children: integer (nullable = true)
 |-- income: double (nullable = true)
 |-- income_type: string (nullable = true)
 |-- education: string (nullable = true)
 |-- family_status: string (nullable = true)
 |-- housing_type: string (nullable = true)
 |-- age_in_days: integer (nullable = true)
 |-- employment_in_days: integer (nullable = true)
 |-- work_phone: integer (nullable = true)
 |-- phone: integer (nullable = true)
 |-- email: integer (nullable = true)
 |-- occupation_type: string (nullable = true)
 |-- family_members: integer (nullable = true)
 |-- age: long (nullable = true)
 |-- employed: long (nullable = true)



In [24]:
#Categorical Data
df_app.select('income_type').distinct().show()
df_app.select('education').distinct().show()
df_app.select('family_status').distinct().show()
df_app.select('housing_type').distinct().show()
df_app.select('occupation_type').distinct().show()

+--------------------+
|         income_type|
+--------------------+
|             Student|
|Commercial associate|
|       State servant|
|             Working|
|           Pensioner|
+--------------------+

+--------------------+
|           education|
+--------------------+
|     Academic degree|
|   Incomplete higher|
|Secondary / secon...|
|     Lower secondary|
|    Higher education|
+--------------------+

+--------------------+
|       family_status|
+--------------------+
|           Separated|
|             Married|
|Single / not married|
|               Widow|
|      Civil marriage|
+--------------------+

+-------------------+
|       housing_type|
+-------------------+
|  House / apartment|
|Municipal apartment|
|    Co-op apartment|
|   Rented apartment|
|   Office apartment|
|       With parents|
+-------------------+

+--------------------+
|     occupation_type|
+--------------------+
|            Managers|
|            HR staff|
|      Medicine staff|
|         Account

In [25]:
from pyspark.sql.functions import col, countDistinct
from pyspark.sql.types import StringType

string_columns = [field.name for field in df_app.schema.fields if field.dataType == StringType()]

# Create a DataFrame to hold the column names and their counts of unique values
unique_counts = df_app.select([countDistinct(col(c)).alias(c) for c in string_columns])

# Convert the above DataFrame to have two columns: Column_Name and Num_Unique
# First, create a single column DataFrame where each row has the format (ColumnName, Count)
unique_counts = unique_counts.selectExpr("stack(" + str(len(string_columns)) + ", " + 
                                         ", ".join([f"'{col}', {col}" for col in string_columns]) + 
                                         ") as (Column_Name, Num_Unique)")

# Sort by Num_Unique
unique_counts = unique_counts.sort("Num_Unique")

# Show the results
unique_counts.show()

+---------------+----------+
|    Column_Name|Num_Unique|
+---------------+----------+
|    income_type|         5|
|      education|         5|
|  family_status|         5|
|   housing_type|         6|
|occupation_type|        18|
+---------------+----------+



In [26]:
open_month = df_credit.groupBy("ID").agg(min("MONTHS_BALANCE").alias("begin_month"))

# Merge with the data DataFrame
customer_data = df_app.join(open_month, on="ID", how="left")

# Show the schema of the resulting DataFrame to confirm
customer_data.select(['id', 'begin_month', 'gender', 'own_Car']).show()

+-------+-----------+------+-------+
|     id|begin_month|gender|own_Car|
+-------+-----------+------+-------+
|5008804|         -6|     0|      1|
|5008805|         -6|     0|      1|
|5008806|         -6|     0|      1|
|5008808|         -4|     1|      0|
|5008809|       NULL|     1|      0|
|5008810|         -6|     1|      0|
|5008811|         -6|     1|      0|
|5008812|         -6|     1|      0|
|5008813|         -6|     1|      0|
|5008814|         -6|     1|      0|
|5008815|         -5|     0|      1|
|5112956|         -6|     0|      1|
|6153651|       NULL|     0|      1|
|5008819|       NULL|     0|      1|
|5008820|       NULL|     0|      1|
|5008821|         -6|     0|      1|
|5008822|       NULL|     0|      1|
|5008823|         -6|     0|      1|
|5008824|         -3|     0|      1|
|5008825|         -6|     1|      1|
+-------+-----------+------+-------+
only showing top 20 rows



In [27]:
#Compact housing type
housing_type = {'House / apartment' : 'House / apartment',
                   'With parents': 'With parents',
                    'Municipal apartment' : 'House / apartment',
                    'Rented apartment': 'House / apartment',
                    'Office apartment': 'House / apartment',
                    'Co-op apartment': 'House / apartment'}

def map_housing_type(col_name):
    
    expression = when(col(col_name) == 'With parents', housing_type['With parents'])
    for key, value in housing_type.items():
        if key != 'With parents': 
            expression = expression.when(col(col_name) == key, value)
    return expression.otherwise(col(col_name)) 

customer_data = customer_data.withColumn("Housing_Type", map_housing_type("Housing_Type"))

customer_data.select('housing_type').distinct().show()

+-----------------+
|     housing_type|
+-----------------+
|House / apartment|
|     With parents|
+-----------------+



In [28]:
#Compact education type
education_type = {
    'Secondary / secondary special': 'secondary',
    'Lower secondary': 'secondary',
    'Higher education': 'Higher education',
    'Incomplete higher': 'Higher education',
    'Academic degree': 'Academic degree'
}

def map_education_type(col_name):
    expression = when(col(col_name) == 'Secondary / secondary special', education_type['Secondary / secondary special'])
    for key, value in education_type.items():
        if key != 'Secondary / secondary special':  
            expression = expression.when(col(col_name) == key, value)
    return expression.otherwise(col(col_name)) 

customer_data = customer_data.withColumn("Education", map_education_type("Education"))
customer_data.select('Education').distinct().show()

+----------------+
|       Education|
+----------------+
| Academic degree|
|       secondary|
|Higher education|
+----------------+



In [29]:
#Compact income type
income_type = {
    'Commercial associate': 'Working',
    'State servant': 'Working',
    'Working': 'Working',
    'Pensioner': 'Pensioner',
    'Student':'Student'
}

def map_income_type(col_name):
    
    expression = when(col(col_name) == 'Working', income_type['Working'])  
    for key, value in income_type.items():
        if key != 'Working': 
            expression = expression.when(col(col_name) == key, value)
    return expression.otherwise(col(col_name))

# Update the DataFrame
customer_data = customer_data.withColumn("Income_Type", map_income_type("Income_Type"))

# Show the updated results to verify the changes
customer_data.select('Income_Type').distinct().show()

+-----------+
|Income_Type|
+-----------+
|    Student|
|    Working|
|  Pensioner|
+-----------+



In [30]:
customer_data = customer_data.withColumn(
    "In_Relationship",
    when(
        (col("Family_Status") == "Civil marriage") | (col("Family_Status") == "Married"), 1
    ).otherwise(0)
)

# Show the updated results to verify the changes
customer_data.select("Family_Status", "In_Relationship").show(truncate=False)

+--------------------+---------------+
|Family_Status       |In_Relationship|
+--------------------+---------------+
|Civil marriage      |1              |
|Civil marriage      |1              |
|Married             |1              |
|Single / not married|0              |
|Single / not married|0              |
|Single / not married|0              |
|Single / not married|0              |
|Separated           |0              |
|Separated           |0              |
|Separated           |0              |
|Married             |1              |
|Married             |1              |
|Married             |1              |
|Married             |1              |
|Married             |1              |
|Married             |1              |
|Married             |1              |
|Married             |1              |
|Married             |1              |
|Married             |1              |
+--------------------+---------------+
only showing top 20 rows



In [31]:
customer_data.printSchema()

root
 |-- id: integer (nullable = true)
 |-- gender: integer (nullable = false)
 |-- own_car: integer (nullable = false)
 |-- own_property: integer (nullable = false)
 |-- children: integer (nullable = true)
 |-- income: double (nullable = true)
 |-- Income_Type: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- family_status: string (nullable = true)
 |-- Housing_Type: string (nullable = true)
 |-- age_in_days: integer (nullable = true)
 |-- employment_in_days: integer (nullable = true)
 |-- work_phone: integer (nullable = true)
 |-- phone: integer (nullable = true)
 |-- email: integer (nullable = true)
 |-- occupation_type: string (nullable = true)
 |-- family_members: integer (nullable = true)
 |-- age: long (nullable = true)
 |-- employed: long (nullable = true)
 |-- begin_month: integer (nullable = true)
 |-- In_Relationship: integer (nullable = false)



In [33]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

data_to_plot = customer_data.select("Income", "Age", "Employed", "Household_Size").toPandas()

# Create subplot structure
fig = make_subplots(rows=2, cols=2, start_cell="bottom-left",
                    subplot_titles=("Income", "Age", "Employed", "Household Size"))

# Add box plots
fig.add_trace(go.Box(x=data_to_plot['Income'], name='Income', boxmean='sd'), row=1, col=1)
fig.add_trace(go.Box(x=data_to_plot['Age'], name='Age', boxmean='sd'), row=1, col=2)
fig.add_trace(go.Box(x=data_to_plot['Employed'], name='Experience', boxmean='sd'), row=2, col=1)
fig.add_trace(go.Box(x=data_to_plot['Household_Size'], name='Household Size', boxmean='sd'), row=2, col=2)

# Display the figure
fig.show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `Household_Size` cannot be resolved. Did you mean one of the following? [`Housing_Type`, `children`, `Income_Type`, `email`, `employed`].;
'Project [Income#2040, Age#536L, Employed#601L, 'Household_Size]
+- Project [id#17, gender#1960, own_car#1980, own_property#2000, children#2020, income#2040, Income_Type#2552, Education#2524, family_status#2100, Housing_Type#2496, age_in_days#2277, employment_in_days#2297, work_phone#2159, phone#2179, email#2199, occupation_type#976, family_members#2219, age#536L, employed#601L, begin_month#2441, CASE WHEN ((Family_Status#2100 = Civil marriage) OR (Family_Status#2100 = Married)) THEN 1 ELSE 0 END AS In_Relationship#2580]
   +- Project [id#17, gender#1960, own_car#1980, own_property#2000, children#2020, income#2040, CASE WHEN (Income_Type#2060 = Working) THEN Working WHEN (Income_Type#2060 = Commercial associate) THEN Working WHEN (Income_Type#2060 = State